In [90]:
import ipynb.fs.full.preProcessing as preProcess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re
import json
import seaborn as sns

DECISION TREE MODEL:

FULL SET REGRESSOR:
Features: 

All 79 features from communities (with 2 extra features for distance), housing price at t-1, egm at t-1, offence rate at t-1 => Predict the offence rate at t

(t in [2015, 2016] to ensure the communities data are still relevant)

Use continuous output directly

In [91]:
# Preparing the frame for training model
offences = preProcess.cleanedOffences
offences = offences.drop(columns=["Year ending", "Police Region", "Offence Count"])

communities = preProcess.cleanedCommunities
communities["Distance from Melbourne"] = np.sqrt(communities["Location_x"]**2 + communities["Location_y"]**2)
communities["Direction from Melbourne"] = np.arctan(communities["Location_x"]/ communities["Location_y"])

housingPrices = preProcess.cleanedHousingPrice
egm = preProcess.cleanedEgm


In [92]:
from sklearn.preprocessing import OneHotEncoder

modelData = offences[(offences["Year"] == 2015) | (offences["Year"] == 2016)]
modelData = pd.merge(left=modelData, right=communities, how="left", on="LGA")

for idx, row in modelData.iterrows():
    modelData.loc[idx, "Last Offence Rate"] = offences[(offences["LGA"] == row["LGA"]) 
                                            & (offences["Year"] == row["Year"] - 1)]["Rate per 100,000 population"].iloc[0]
    modelData.loc[idx, "Last Housing Price"] = housingPrices.loc[row["LGA"]][str(row["Year"] - 1)]
    modelData.loc[idx, "Last EGM"] = egm.loc[row["LGA"]][str(row["Year"] - 1)]


In [93]:

# One hot the LGA - Use inverse transform from the encoder to get back the lga

# oneHotEncoder = OneHotEncoder(handle_unknown='ignore').fit(np.array(modelData["LGA"]).reshape(-1, 1))
# oneHotLGA = pd.DataFrame(oneHotEncoder
#                          .transform(np.array(modelData["LGA"]).reshape(-1, 1))
#                          .toarray())
# modelData = modelData.join(oneHotLGA)
modelData = modelData.drop(columns=["LGA"])

modelData.columns = modelData.columns.astype('str')

modelData

,Year,"Rate per 100,000 population",Travel time to GPO (minutes),Area (km^2),ARIA+ (min),ARIA+ (max),ARIA+ (avg),Commercial (km^2),Industrial (km^2),Residential (km^2),...,Number of Dwellings,Population,Location_x,Location_y,ABS remoteness category,Distance from Melbourne,Direction from Melbourne,Last Offence Rate,Last Housing Price,Last EGM
0,2016,7903.7,20.289781,84.535123,0.000000,0.000000,0.000000,0.008991,0.021552,0.605656,...,59303.0,156425.0,10.077340,9.265546,0,13.689526,0.827342,7317.1,781832.825595,5.451265e+07
1,2016,9375.1,19.419507,128.349256,0.000000,0.000000,0.000000,0.016543,0.213052,0.474939,...,71298.0,207607.0,-13.317025,7.863303,0,15.465274,-1.037414,9572.8,485168.945244,1.416092e+08
2,2016,10864.1,14.544640,45.693749,0.000000,0.000000,0.000000,0.029054,0.067348,0.697212,...,54724.0,132982.0,6.616863,5.985467,0,8.922370,0.835458,10093.2,785645.012514,8.385740e+07
3,2016,7166.8,16.202194,65.887204,0.000000,0.000000,0.000000,0.010994,0.107162,0.366402,...,36218.0,90428.0,-10.824519,-4.394903,0,11.682696,1.185117,7523.4,680123.763681,4.643098e+07
4,2016,11420.0,23.587200,599.996103,0.023828,0.037001,0.029642,0.016390,0.026318,0.094048,...,62098.0,190907.0,-8.386907,17.687785,0,19.575443,-0.442766,10339.8,538186.357972,1.049440e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,2015,11385.6,384.960766,23359.313312,2.687931,3.482185,3.087847,0.000074,0.000461,0.001773,...,23533.0,54030.0,-280.304300,343.714443,2,443.520145,-0.684130,11044.5,187437.500000,2.470024e+07
108,2015,6857.1,58.368445,2142.863230,0.331739,0.880712,0.598316,0.000394,0.002019,0.020584,...,18640.0,47165.0,-63.629211,23.520312,1,67.837170,-1.216727,6183.6,339500.000000,1.030988e+07
109,2015,9876.3,179.410340,6720.196354,2.135649,2.837918,2.452597,0.000128,0.000423,0.002905,...,7094.0,13042.0,-179.798887,102.227446,2,206.828650,-1.053811,7947.7,158750.000000,1.035065e+07
110,2015,5198.3,94.497165,1652.413568,0.512327,0.704602,0.606605,0.000191,0.000432,0.016410,...,13533.0,23987.0,-74.669919,-62.823737,1,97.582881,0.871344,4886.7,645600.000000,2.855820e+06


In [101]:
# Use hold out for now
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Models: https://www.analyticsvidhya.com/blog/2021/09/adaboost-algorithm-a-complete-guide-for-beginners/
# AdaBoost model (TODO: Recheck source): Train an initial tree with uniform weight for all points.
# Then, for subsequent iterations, change the weights so more weights are given for
# the points with errors.
# Then, random selection based on the normalized probability weights for the data points
# to construct a new datasets => Train a new tree on top of it 
# At the end, with 300 iterations, around 300 decision trees are built and used for regression
decisionTreeRegressor = DecisionTreeRegressor(max_depth=5)
adaboostRegressor = AdaBoostRegressor(
    DecisionTreeRegressor(max_depth=5), n_estimators=400, random_state=1301266
)

X = modelData.drop(columns=["Rate per 100,000 population", "Year"])
y = modelData["Rate per 100,000 population"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

decisionTreeRegressor.fit(X_train, y_train)
adaboostRegressor.fit(X_train, y_train)

y_predict = decisionTreeRegressor.predict(X_test)
y_predict2 = adaboostRegressor.predict(X_test)
y_null_predict = [y_train.mean()] * len(X_test)
y_null_2 = X_test["Last Offence Rate"]

mse_regressor = mean_squared_error(y_test, y_predict)
mse_ada = mean_squared_error(y_test, y_predict2)
mse_null = mean_squared_error(y_test, y_null_predict)
mse_null_2 = mean_squared_error(y_test, y_null_2)


print("MSE Error of decision tree regressor: ", mse_regressor)
print("MSE Ada: ", mse_ada)
print("MSE Error of null mean model: ", mse_null)
print("MSE Error of null last model: ", mse_null_2)
print("Ratio regressor/null1: ", mse_regressor/mse_null)
print("Ratio regressor/null2: ", mse_regressor/mse_null_2)
print("Ratio ada/null2: ", mse_ada/mse_null_2)



MSE Error of decision tree regressor:  1085119.4022023077
MSE Ada:  374886.65632375283
MSE Error of null mean model:  5276531.472541766
MSE Error of null last model:  846750.3317647058
Ratio regressor/null1:  0.2056501335866368
Ratio regressor/null2:  1.2815104541392015
Ratio ada/null2:  0.44273576550298416


In [96]:
# K-fold, since the above is pretty random
from sklearn.model_selection import KFold

n = 100

nf_CV = KFold(n_splits=n, shuffle=True, random_state=42)
mses = {
    "null": [],
    "null2": [],
    "ada": [],
    "regressor": []
}

for train_idx, test_idx in nf_CV.split(X):
    # train-test split
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    y_predict = decisionTreeRegressor.predict(X_test)
    y_predict2 = adaboostRegressor.predict(X_test)
    y_null_predict = [y_train.mean()] * len(X_test)
    y_null_2 = X_test["Last Offence Rate"]

    mse_regressor = mean_squared_error(y_test, y_predict)
    mse_ada = mean_squared_error(y_test, y_predict2)
    mse_null = mean_squared_error(y_test, y_null_predict)
    mse_null_2 = mean_squared_error(y_test, y_null_2)

    mses["ada"].append(mse_ada)
    mses["regressor"].append(mse_regressor)
    mses["null"].append(mse_null)
    mses["null2"].append(mse_null_2)

print(np.mean(mses["ada"])/np.mean(mses["null2"]))
print(np.mean(mses["ada"])/np.mean(mses["null"]))
print(np.mean(mses["regressor"])/np.mean(mses["null2"]))
print(np.mean(mses["regressor"])/np.mean(mses["null"]))

    

0.3183149928871794
0.01943041560907209
0.6351113654567974
0.03876813239910659
